<a href="https://colab.research.google.com/github/Rohitd922/Operation-Research-projects/blob/master/sudoku_generator_and_mip_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 56.3 MB/s eta 0:00:00


In [6]:
import os
import random
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

## Create a valid sudoku puzzle using recursion and backtracking

In [7]:
### Create a new sudoku puzzle

def find_empty_cell(board):
    """
    Returns (row, col) of an empty cell (value=0) or None if no empty cell.
    """
    for r in range(9):
        for c in range(9):
            if board[r][c] == 0:
                return (r, c)
    return None

def is_safe(board, row, col, num):
    """
    Checks if placing `num` at board[row][col] is valid under Sudoku constraints.
    """
    # Row check
    if num in board[row]:
        return False

    # Column check
    for r in range(9):
        if board[r][col] == num:
            return False

    # 3x3 subgrid check
    box_r = (row // 3) * 3
    box_c = (col // 3) * 3
    for rr in range(box_r, box_r + 3):
        for cc in range(box_c, box_c + 3):
            if board[rr][cc] == num:
                return False

    return True

def solve_sudoku(board, limit_solutions=2):
    """
    Backtracking solver that can also detect if there's more than 1 solution.
      - `limit_solutions=2` means: if it finds 2 solutions, it can stop
        (we only need to know if there's >1 solution).
    Returns the number of solutions found (capped at `limit_solutions`).
    """
    find = find_empty_cell(board)
    if not find:
        # No empty spaces => exactly 1 valid solution found
        return 1
    row, col = find

    solutions_found = 0

    for num in range(1, 10):
        if is_safe(board, row, col, num):
            board[row][col] = num
            result = solve_sudoku(board, limit_solutions)
            board[row][col] = 0  # backtrack

            solutions_found += result
            if solutions_found >= limit_solutions:
                # We found 2 or more solutions, so we can stop searching
                break

    return solutions_found

def generate_full_solution(board):
    """
    Fills an empty (9x9) board with a valid, complete Sudoku solution (randomly).
    """
    find = find_empty_cell(board)
    if not find:
        # Board is fully filled
        return True
    row, col = find

    digits = list(range(1, 10))
    random.shuffle(digits)  # shuffle to get a random solution each time

    for num in digits:
        if is_safe(board, row, col, num):
            board[row][col] = num
            if generate_full_solution(board):
                return True
            board[row][col] = 0  # backtrack

    return False

def puzzle_has_unique_solution(board):
    """
    Returns True if 'board' has exactly one solution; otherwise False.
    """
    # Make a copy since we modify the board during solving
    temp_board = [row[:] for row in board]
    return (solve_sudoku(temp_board, limit_solutions=2) == 1)

def generate_sudoku_puzzle(empty_cells=40):
    """
    Creates a Sudoku puzzle with about 'empty_cells' removed clues
    and exactly 1 solution.
      - Start by generating a full valid solution.
      - Then try removing random cells while checking uniqueness.
      - Return the puzzle as a 9x9 list of lists (0 = empty).
    """
    # Step 1: Generate a complete (random) valid solution
    board = [[0]*9 for _ in range(9)]
    generate_full_solution(board)

    # Step 2: Remove clues randomly, checking uniqueness
    all_positions = [(r, c) for r in range(9) for c in range(9)]
    random.shuffle(all_positions)

    removed_count = 0
    for (r, c) in all_positions:
        if removed_count >= empty_cells:
            break
        backup = board[r][c]
        board[r][c] = 0  # try removing

        # Check if the puzzle still has a unique solution
        if puzzle_has_unique_solution(board):
            removed_count += 1
        else:
            # revert removal
            board[r][c] = backup

    return board

def print_puzzle(puzzle):
    """
    Prints the Sudoku puzzle in a simple grid form.
    """
    for r in range(9):
        if r % 3 == 0 and r != 0:
            print("-"*21)
        row_str = ""
        for c in range(9):
            if c % 3 == 0 and c != 0:
                row_str += "| "
            val = puzzle[r][c]
            row_str += f"{val if val != 0 else '.'} "
        print(row_str)

puzzle = generate_sudoku_puzzle(empty_cells=40)
print_puzzle(puzzle)


4 . . | . 1 . | 7 . . 
6 . 9 | . 3 8 | 4 . 1 
1 . . | 9 . 6 | . . 8 
---------------------
. . 7 | . . 5 | 6 1 . 
. . . | . . . | 8 4 7 
8 4 6 | . . 3 | 2 9 5 
---------------------
9 6 . | 8 . . | 1 3 4 
7 . 4 | . . 1 | 5 . . 
5 . . | 3 2 4 | . . . 


## Use gurobi.py model to solve the model

In [8]:
## Create a Gurobi model
sudoku = gp.Model("sudoku")

Restricted license - for non-production use only - expires 2026-11-23


In [9]:
vars = sudoku.addVars(9, 9, 9, vtype=gp.GRB.BINARY, name="vars")

In [10]:
#### fix the variables to the sudoku generated values

for r in range(9):
    for c in range(9):
        if puzzle[r][c] != 0:
            sudoku.addConstr(vars[r, c, puzzle[r][c]-1] == 1, name=f"fixed_{r}_{c}_{puzzle[r][c]}")

In [11]:
#### cell should take 1 value
for r in range(9):
  for c in range(9):
    sudoku.addConstr(gp.quicksum(vars[r,c,k] for k in range(9)) == 1, name=f"depth_sum_{r}_{c}")

In [12]:
#### row sum should be equal to 1
for r in range(9):
  for k in range(9):
    sudoku.addConstr(gp.quicksum(vars[r,c,k] for c in range(9)) == 1, name=f"row_sum_{r}_{k}")

In [13]:
#### column sum should be equal to 1

for c in range(9):
  for k in range(9):
    sudoku.addConstr(gp.quicksum(vars[r,c,k] for r in range(9))==1, name=f"col_sum_{c}_{k}")

In [14]:
#### in submatrix of size 3x3
for r in (range(0,9,3)):
  for c in (range(0,9,3)):
    for k in range(9):
      sudoku.addConstr(gp.quicksum(vars[rr,cc,k] for rr in range(r,r+3) for cc in range(c,c+3))==1, name=f"submatrix_{r}_{c}_{k}")

In [15]:
#### solve the sudoku

sudoku.params.TimeLimit = 60
sudoku.optimize()

Set parameter TimeLimit to value 60
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
TimeLimit  60

Optimize a model with 365 rows, 729 columns and 2957 nonzeros
Model fingerprint: 0xb46b1935
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 365 rows and 729 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [16]:
#### print the values of vars in form of a 3d matrix

vars[0,0,3].X

### initialize an empty np array of size 9x9
solution = np.zeros((9,9), dtype=int)

for r in range(9):
  for c in range(9):
    for k in range(9):
      if vars[r,c,k].X == 1:
        solution[r,c] = k+1
        break

print_puzzle(solution)


4 8 3 | 5 1 2 | 7 6 9 
6 2 9 | 7 3 8 | 4 5 1 
1 7 5 | 9 4 6 | 3 2 8 
---------------------
2 9 7 | 4 8 5 | 6 1 3 
3 5 1 | 2 6 9 | 8 4 7 
8 4 6 | 1 7 3 | 2 9 5 
---------------------
9 6 2 | 8 5 7 | 1 3 4 
7 3 4 | 6 9 1 | 5 8 2 
5 1 8 | 3 2 4 | 9 7 6 


In [17]:
print_puzzle(np.array(puzzle))

4 . . | . 1 . | 7 . . 
6 . 9 | . 3 8 | 4 . 1 
1 . . | 9 . 6 | . . 8 
---------------------
. . 7 | . . 5 | 6 1 . 
. . . | . . . | 8 4 7 
8 4 6 | . . 3 | 2 9 5 
---------------------
9 6 . | 8 . . | 1 3 4 
7 . 4 | . . 1 | 5 . . 
5 . . | 3 2 4 | . . . 
